In [1]:
import pandas as pd
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5


In [3]:
df.dropna(inplace=True)
df = df[df['Rating'] <= 5]

In [4]:
neg_reviews = list(df['Review'][df['Rating'] < 4][:55])
neg_train = neg_reviews[:50]
neg_test = neg_reviews[50:]

pos_reviews = list(df['Review'][df['Rating'] >= 4][:55])
pos_train = pos_reviews[:50]
pos_test = pos_reviews[50:]

X_train = pos_train + neg_train
X_test = pos_test + neg_test
y_train = [1] * 50 + [0] * 50
y_test = [1] * 5 + [0] * 5

In [5]:
m = MorphAnalyzer()

In [6]:
sw = stopwords.words('russian')

def lemmatize(reviews_list):
    lemmas_w_noise = [m.parse(word)[0].normal_form for review in reviews_list
                      for sent in nltk.sent_tokenize(review)
                      for word in nltk.word_tokenize(sent)]
    lemmas = [lemma for lemma in lemmas_w_noise
              if lemma not in punctuation and lemma not in sw]
    return lemmas

In [7]:
def lemmatize_text(review):
    lemmas_w_noise = [m.parse(word)[0].normal_form 
                      for sent in nltk.sent_tokenize(review)
                      for word in nltk.word_tokenize(sent)]
    lemmas = [lemma for lemma in lemmas_w_noise
              if lemma not in punctuation and lemma not in sw]
    return lemmas

In [8]:
def most_common(lemmas):
    counted = Counter(lemmas)
    new_lemmas = set({k:v for k,v in counted.items() if v > 2 })
    return new_lemmas

In [9]:
neg_lemmas = lemmatize(neg_train)
neg_lemmas = most_common(neg_lemmas)

pos_lemmas = lemmatize(pos_train)
pos_lemmas = most_common(pos_lemmas)

In [10]:
pos_lemmas.difference_update(neg_lemmas)  # леммы, встречающиеся только в положительных отзывах
neg_lemmas.difference_update(pos_lemmas)  # и только в отрицательных

In [11]:
def review_type(text):
    lemmas = set(lemmatize_text(text))
    if len(lemmas.intersection(pos_lemmas)) >= len(lemmas.intersection(neg_lemmas)):
        return 1
    else:
        return 0

In [12]:
y_pred_tr = [review_type(r) for r in X_train]
y_pred_te = [review_type(r) for r in X_test]

In [13]:
train_acc = accuracy_score(y_train, y_pred_tr)
test_acc = accuracy_score(y_test, y_pred_te)

In [14]:
print(train_acc)
print(test_acc)

0.54
0.3


## Способы улучшить программу
1. Выделить коллокации и частотные словосочетания и подавать их в программу
2. Провести named entity recognition
3. Сделать POS-теггинг, чтобы частично снять омонимию
4. Также можно векторизовать тексты и воспользоваться инструментами машинного обучения

Попробую выделить коллокации и тем самым улучшить качество работы программы

In [15]:
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

finder = BigramCollocationFinder.from_words(neg_lemmas)
bgm = BigramAssocMeasures()
score = bgm.mi_like
collocations = {' '.join(bigram): pmi for bigram, pmi in finder.score_ngrams(score)}

In [21]:
colloc = [key for key, value in collocations.items() if value == 1]

In [22]:
colloc

["'' месяц",
 '... менять',
 '1 музыка',
 '100 рука',
 '12 разговор',
 '2 3',
 '3 написать',
 '4 100',
 '4055 советовать',
 '5 батарейка',
 '6 возможно',
 '7 держать',
 '`` сенсор',
 'apple 4',
 'asus думать',
 'duo купить',
 'energy брать',
 'fly заменить',
 'prestigio долго',
 'wi-fi аккумулятор',
 'аккумулятор плохо',
 'акселерометр дизайн',
 'андроид плохой',
 'аппарат прийтись',
 'батарейка постоянно',
 'батарея год',
 'больший назад',
 'большой чехол',
 'брак 2',
 'брать новый',
 'быстро отзыв',
 'весь делать',
 'взять ``',
 'видео андроид',
 'возможно свой',
 'вообще зависание',
 'время работа',
 'всё компания',
 'выход простой',
 'главный больший',
 'глючить 4055',
 'год начать',
 'гораздо использовать',
 'данный energy',
 'делать телефон',
 'день задний',
 'деньга ремонт',
 'держать замена',
 'дизайн который',
 'довольный 6',
 'долго 7',
 'думать видео',
 'ещё писать',
 'зависание тянуть',
 'задний клавиша',
 'замена из-за',
 'заменить игра',
 'заряд режим',
 'звонить хватить'

In [23]:
len(colloc)

179

In [24]:
colloc = [key for key, value in collocations.items() if value > 1]

In [25]:
len(colloc)

0

Так как все полученные коллокации имеют значение pmi не больше 1, то ничего, к сожалению, не вышло и нужны другие инструменты